In [1]:
import sympy


input_file = "data/input.txt"

START = 200000000000000
END = 400000000000000

def parse_coordinates(line):
    pos, velo = line.split("@")
    pos = [int(x) for x in pos.split(",")]
    velo = [int(x) for x in velo.split(",")]
    return pos, velo

def to_line(pos, velo):
    a = velo[1] / velo[0]
    b = pos[1] - (a * pos[0])
    xrange = [-1e30, 1e30]
    yrange = [-1e30, 1e30]
    if velo[0] >= 0:
        xrange[0] = pos[0]
    else:
        xrange[1] = pos[0]
    if velo[1] >= 0:
        yrange[0] = pos[1]
    else:
        yrange[1] = pos[1]
    return a, b, xrange, yrange

def get_intersect_point(line1, line2):
    a1, b1, xrange1, yrange1 = line1
    a2, b2, xrange2, yrange2 = line2
    xstart = max(xrange1[0], xrange2[0])
    xend = min(xrange1[1], xrange2[1])
    ystart = max(yrange1[0], yrange2[0])
    yend = min(yrange1[1], yrange2[1])

    if (a1 - a2) == 0:
        return -1e30, -1e30
    x = (b2 - b1) / (a1 - a2)
    y = a1 * x + b1
    if xstart <= x <= xend and ystart <= y <= yend:
        return x, y
    return -1e30, -1e30

def solve_collision(rocks):
    ans_x = sympy.Symbol("x")
    ans_y = sympy.Symbol("y")
    ans_z = sympy.Symbol("z")
    ans_vx = sympy.Symbol("vx")
    ans_vy = sympy.Symbol("vy")
    ans_vz = sympy.Symbol("vz")
    equations = []
    time_symbols = []

    for rock_id, rock in enumerate(rocks[:3]):
        x, y, z = rock[0]
        vx, vy, vz = rock[1]
        time = sympy.Symbol(f't{rock_id}')
        eq_x = (ans_x + ans_vx * time) - (x - vx * time)
        eq_y = (ans_y + ans_vy * time) - (y - vy * time)
        eq_z = (ans_z + ans_vz * time) - (z - vz * time)
        equations.append(eq_x)
        equations.append(eq_y)
        equations.append(eq_z)
        time_symbols.append(time)
    
    all_symbols = [ans_x, ans_y, ans_z, ans_vx, ans_vy, ans_vz, *time_symbols]
    result = sympy.solve_poly_system(equations, *all_symbols)
    return result[0]

with open(input_file, 'r') as f:
    lines = [l.strip() for l in f.readlines()]
    rocks = [parse_coordinates(l) for l in lines]
    line_equations = []
    for pos, velo in rocks:
        line = to_line(pos, velo)
        line_equations.append(line)

    ans1 = 0
    for i in range(len(line_equations)):
        for j in range(i, len(line_equations)):
            line1 = line_equations[i]
            line2 = line_equations[j]
            if line1 == line2:
                continue
            x, y = get_intersect_point(line1, line2)
            if START <= x <= END and START <= y <= END:
                ans1 += 1

    x, y, z, *_ = solve_collision(rocks)
    ans2 = x + y + z

    print(f"{ans1 = }")
    print(f"{ans2 = }")

ans1 = 18651
ans2 = 546494494317645
